<a href="https://colab.research.google.com/github/juli-ter/Coursework4/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels.api as sm
import plotly.express as px
import datetime as dt
import warnings
import random
import warnings
warnings.filterwarnings('ignore')

import statsmodels.formula.api as smf
import graphviz as gr
%matplotlib inline

raw_path = '/content/drive/MyDrive/Marketing/data/'


In [4]:
df = pd.read_csv(raw_path + 'train_eda_labeled.csv', index_col=False)

In [5]:
df

,id,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,family,class,perishable,dcoilwtico,is_holiday,day_of_week,day_of_year,year,month,day,label
0,21657671,1,115611,46.00,0.0,Quito,Pichincha,D,13,GROCERY I,1014,0,99.69,1,1,91,2014,4,1,3
1,23349772,1,115611,13.00,0.0,Quito,Pichincha,D,13,GROCERY I,1014,0,99.74,0,0,125,2014,5,5,3
2,24087515,1,115611,9.00,0.0,Quito,Pichincha,D,13,GROCERY I,1014,0,102.80,0,1,140,2014,5,20,3
3,29022175,1,115611,7.00,0.0,Quito,Pichincha,D,13,GROCERY I,1014,0,95.54,0,3,226,2014,8,14,3
4,36515701,1,115611,10.00,0.0,Quito,Pichincha,D,13,GROCERY I,1014,0,67.30,0,2,337,2014,12,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513480,41547484,51,1239986,26.31,1.0,Guayaquil,Guayas,A,17,POULTRY,2420,1,49.95,0,4,51,2015,2,20,0
3513481,93890355,51,1239986,2594.00,1.0,Guayaquil,Guayas,A,17,POULTRY,2420,1,50.35,0,4,288,2016,10,14,0
3513482,100163926,51,1239986,2656.20,1.0,Guayaquil,Guayas,A,17,POULTRY,2420,1,51.93,0,4,351,2016,12,16,0
3513483,102944669,51,1239986,4147.00,1.0,Guayaquil,Guayas,A,17,POULTRY,2420,1,52.36,0,4,13,2017,1,13,0


In [ ]:
# date = pd.to_datetime(df[['year', 'month', 'day']])
# df['date'] = date


In [8]:
result = smf.ols('unit_sales ~ C(onpromotion)', data=df).fit()
result.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,7.3055,0.012,603.628,0.000,7.282,7.329
C(onpromotion)[T.1.0],5.9033,0.042,141.353,0.000,5.821,5.985


In [19]:
result = smf.ols('np.log(unit_sales) ~ C(onpromotion)', data=df).fit()
result.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,1.3413,0.001,2271.926,0.000,1.340,1.342
C(onpromotion)[T.1.0],0.4391,0.002,215.537,0.000,0.435,0.443


promo increased unit_sales by nearly 6 units on average

7.3 units is bought on average without promo

In [7]:
(df
 .groupby("onpromotion")
 ["unit_sales"]
 .mean())

onpromotion
0.0     7.305474
1.0    13.208804
Name: unit_sales, dtype: float64

In [17]:
controls = ['city',
 'state',
 'type',
 'family',
 'store_nbr',
 'onpromotion',
 'perishable',
 'is_holiday',
 'cluster',
 'item_nbr', 'dcoilwtico', 'year', 'month', 'day']

In [18]:
# formula

'unit_sales ~ onpromotion +city+state+type+family+store_nbr+onpromotion+perishable+is_holiday+cluster+item_nbr+unit_sales+class+dcoilwtico+year+month+day'

In [21]:
formula = 'unit_sales ~ onpromotion +' + '+'.join(controls)
model = smf.ols('unit_sales ~ onpromotion +state+item_nbr+dcoilwtico+C(is_holiday)+cluster+C(label)', data=df).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,8.7644,0.080,109.100,0.000,8.607,8.922
state[T.Bolivar],1.7594,0.115,15.329,0.000,1.534,1.984
state[T.Chimborazo],2.7363,0.122,22.467,0.000,2.498,2.975
state[T.Cotopaxi],3.5632,0.106,33.599,0.000,3.355,3.771
state[T.El Oro],0.1866,0.077,2.419,0.016,0.035,0.338
state[T.Esmeraldas],1.4775,0.114,12.939,0.000,1.254,1.701
state[T.Guayas],0.4973,0.057,8.776,0.000,0.386,0.608
state[T.Imbabura],3.8275,0.122,31.253,0.000,3.587,4.068
state[T.Loja],-0.5193,0.093,-5.594,0.000,-0.701,-0.337
state[T.Los Rios],0.6756,0.080,8.449,0.000,0.519,0.832


In [23]:
%%capture
!pip install econml
!pip install dowhy

In [24]:
import pickle
import econml
import dowhy

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [40]:
from datetime import date
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df['start'] = pd.to_datetime(date(2014, 4, 1))
df['num_of_day'] = (df['date'] - df['start']).dt.days

In [42]:
model = dowhy.CausalModel(
        data = df,
        treatment= "onpromotion",
        outcome= "unit_sales",
        )

In [30]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                     
──────────────(E[unitₛₐₗₑₛ])
d[onpromotion]              
Estimand assumption 1, Unconfoundedness: If U→{onpromotion} and U→unit_sales then P(unit_sales|onpromotion,,U) = P(unit_sales|onpromotion,)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!



In [38]:
linear_estimate = model.estimate_effect(identified_estimand, 
                                        method_name="backdoor.linear_regression",
                                       control_value=0,
                                       treatment_value=1)
print(linear_estimate) 

*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                     
──────────────(E[unitₛₐₗₑₛ])
d[onpromotion]              
Estimand assumption 1, Unconfoundedness: If U→{onpromotion} and U→unit_sales then P(unit_sales|onpromotion,,U) = P(unit_sales|onpromotion,)

## Realized estimand
b: unit_sales~onpromotion
Target units: ate

## Estimate
Mean value: 5.903329717104124



In [34]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor

In [41]:
df

,id,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,family,...,is_holiday,day_of_week,day_of_year,year,month,day,label,start,date,num_of_day
0,21657671,1,115611,46.00,0.0,Quito,Pichincha,D,13,GROCERY I,...,1,1,91,2014,4,1,3,2014-04-01,2014-04-01,0
1,23349772,1,115611,13.00,0.0,Quito,Pichincha,D,13,GROCERY I,...,0,0,125,2014,5,5,3,2014-04-01,2014-05-05,34
2,24087515,1,115611,9.00,0.0,Quito,Pichincha,D,13,GROCERY I,...,0,1,140,2014,5,20,3,2014-04-01,2014-05-20,49
3,29022175,1,115611,7.00,0.0,Quito,Pichincha,D,13,GROCERY I,...,0,3,226,2014,8,14,3,2014-04-01,2014-08-14,135
4,36515701,1,115611,10.00,0.0,Quito,Pichincha,D,13,GROCERY I,...,0,2,337,2014,12,3,3,2014-04-01,2014-12-03,246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513480,41547484,51,1239986,26.31,1.0,Guayaquil,Guayas,A,17,POULTRY,...,0,4,51,2015,2,20,0,2014-04-01,2015-02-20,325
3513481,93890355,51,1239986,2594.00,1.0,Guayaquil,Guayas,A,17,POULTRY,...,0,4,288,2016,10,14,0,2014-04-01,2016-10-14,927
3513482,100163926,51,1239986,2656.20,1.0,Guayaquil,Guayas,A,17,POULTRY,...,0,4,351,2016,12,16,0,2014-04-01,2016-12-16,990
3513483,102944669,51,1239986,4147.00,1.0,Guayaquil,Guayas,A,17,POULTRY,...,0,4,13,2017,1,13,0,2014-04-01,2017-01-13,1018


In [43]:
dml_estimate = model.estimate_effect(identified_estimand, method_name="backdoor.econml.dml.DML",
                                    control_value = 0,
                                    treatment_value = 1,
                                    target_units = lambda df: df["num_of_day"]>880,  # condition used for CATE
                                    confidence_intervals=False,
                                    method_params={"init_params":{'model_y':GradientBoostingRegressor(),
                                                              'model_t': GradientBoostingRegressor(),
                                                              "model_final":LassoCV(fit_intercept=False), 
                                                              'featurizer':PolynomialFeatures(degree=1, include_bias=False)},
                                                    "fit_params":{}})
print(dml_estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                     
──────────────(E[unitₛₐₗₑₛ])
d[onpromotion]              
Estimand assumption 1, Unconfoundedness: If U→{onpromotion} and U→unit_sales then P(unit_sales|onpromotion,,U) = P(unit_sales|onpromotion,)

## Realized estimand
b: unit_sales~onpromotion | 
Target units: Data subset defined by a function

## Estimate
Mean value: 5.897413817232251
Effect estimates: [[5.89741382]]



In [45]:
dml_estimate = model.estimate_effect(identified_estimand, method_name="backdoor.econml.dml.DML",
                                     control_value = 0,
                                     treatment_value = 1,
                                 target_units = 1,  # condition used for CATE
                                 confidence_intervals=False,
                                method_params={"init_params":{'model_y':GradientBoostingRegressor(),
                                                              'model_t': GradientBoostingRegressor(),
                                                              "model_final":LassoCV(fit_intercept=False), 
                                                              'featurizer':PolynomialFeatures(degree=1, include_bias=True)},
                                               "fit_params":{}})
print(dml_estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                     
──────────────(E[unitₛₐₗₑₛ])
d[onpromotion]              
Estimand assumption 1, Unconfoundedness: If U→{onpromotion} and U→unit_sales then P(unit_sales|onpromotion,,U) = P(unit_sales|onpromotion,)

## Realized estimand
b: unit_sales~onpromotion | 
Target units: 

## Estimate
Mean value: 5.897459264293987
Effect estimates: [[5.89745926]]

